# Topic modelling on the ashley madison profiles.

In [25]:
import pandas as pd
import numpy as np
import logging, gensim
import Image
import os,sys
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords

## I have collected the ashley madison profiles by city, and here's a look at the sample data

In [128]:
profiles = pd.read_csv('ashmad/citydata/state data/bronx_ash.csv')
profiles.columns

Index([u'id', u'createdon', u'createdby', u'updatedon', u'updatedby', u'admin',
       u'status', u'account_type', u'membership_status', u'ad_source',
       u'profile_number', u'nickname', u'first_name', u'last_name', u'street1',
       u'street2', u'city', u'zip', u'state', u'latitude', u'longitude',
       u'country', u'phone', u'work_phone', u'mobile_phone', u'gender', u'dob',
       u'profile_caption', u'profile_ethnicity', u'profile_weight',
       u'profile_height', u'profile_bodytype', u'profile_smoke',
       u'profile_drink', u'profile_initially_seeking', u'profile_relationship',
       u'pref_opento', u'pref_opento_other', u'pref_opento_abstract',
       u'pref_turnsmeon', u'pref_turnsmeon_other', u'pref_turnsmeon_abstract',
       u'pref_lookingfor', u'pref_lookingfor_other',
       u'pref_lookingfor_abstract', u'main_photo', u'security_question',
       u'security_answer'],
      dtype='object')

In [135]:
#I focused on the pref_opento_abstract column. Here's a sample.
list(profiles[profiles['pref_opento_abstract'].notnull()]['pref_opento_abstract'].head())

['I love and respect women and love to see them enjoying it.',
 'Hi i am a 35 year old male looking for a full figured woman in the NY area.',
 "to add on to my profile I may appear innocent on first sight but full of sexual surprises. I'm willing to try anything once and yearning to expand my horizons. My partner is the only one I've ever been with so I have alot to offer by bringing my openess to the table and becoming someone's apprentice. Light skin, light brown eyes, long brown hair and pretty face. Loads of fun with a wild imagination.",
 'WHAT TO TALK AND MAYBE GO FURTHER WITH A ATTRACTIVE WOMAN.',
 'meet someone for fun encounters.']

In [ ]:
path = os.getcwd()+'/ashmad/citydata/state data/'
files=os.listdir(path)

In [126]:
files[:5]

['burlington_ash.csv',
 'washington_ash.csv',
 'san antonio_ash.csv',
 'sacramento_ash.csv',
 'boston_ash.csv']

In [32]:
# for all the city data, take the non-null male profiles. 
male_profile=[]
for a in files:
    try:
        yx=pd.read_csv(path+a)
        male_profile.append(list(yx[(yx['gender']==2) & (yx['pref_opento_abstract'].notnull())]['pref_opento_abstract'].values))
    except:
        pass

#female_profile = list(yx[yx['gender']==1][yx['pref_opento_abstract'].notnull()]['pref_opento_abstract'].values)

In [54]:
#take a peek at all the male profiles
male_profile[2][:5]

['Looking for adventure in Toronto, Canada',
 'young strong and willing to learn from an experienced woman.. but i still got a few tricks of my own.',
 'Just about anythings goes, conventional sex, using sex toys, role play, oral sex, one night stands is cool with me. Looking for that special woman that willing to discover her unlimited supply of orgasms. Have a good since of humor I\xe2\x80\x99m open for threesome or more with safe sex, but I DON\xe2\x80\x99T MENS. I\xe2\x80\x99m very private and very discrete.',
 "I consider myself to be open minded and adventurous but don't have any particular fetishes...I think that a relationship should be fun and I especially like to make sure that my partner is satisfied...I especially like giving oral sex to an appreciative partner.",
 "I hope to meet some new friends who are a little bored with the everyday thing at home, but don't want to change their lifestyle.  Love the antcipation of the first kiss...you can't get that at home, can ya?"]

In [81]:
#Add to list of stopwords
stop=stopwords.words('english')+["lets",'let','thats','that','ill','im','id','doesnt',"couldnt",'wouldnt','shouldnt','dont','didnt','make','would','like','see']

#process the male profiles
yeah = [b for a in male_profile for b in a]
ok=[a.lower().translate(None, string.punctuation).split() for a in yeah ]
bok = [[b for b in a if b not in stop] for a in ok]

# remove single frequency words
from collections import defaultdict
frequency = defaultdict(int)
for text in bok:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in bok]

# Use Gensim to try an LDA model on the male profiles

In [59]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize('/tmp/ash.mm', corpus)
mm = gensim.corpora.MmCorpus('/tmp/ash.mm')

496815

In [92]:
print mm

MmCorpus(496815 documents, 50930 features, 6595565 non-zero entries)


In [239]:
lda = gensim.models.ldamodel.LdaModel(mm, id2word=dictionary,num_topics=20, update_every=1, chunksize=1000, passes=3)

In [242]:
topics=lda.print_topics(20)

In [243]:
topics

[u'0.037*contact + 0.027*read + 0.027*please + 0.025*youre + 0.023*message + 0.021*interested + 0.020*profile + 0.017*member + 0.015*send + 0.015*hot',
 u'0.035*u + 0.032*ask + 0.030*lol + 0.023*wanna + 0.022*hey + 0.018*play + 0.017*love + 0.015*cause + 0.015*hard + 0.014*skinny',
 u'0.048*bed + 0.031*getting + 0.027*big + 0.025*sex + 0.020*super + 0.019*going + 0.018*shy + 0.018*live + 0.017*worth + 0.017*tend',
 u'0.098*fun + 0.082*looking + 0.046*good + 0.041*guy + 0.035*time + 0.022*love + 0.020*someone + 0.019*going + 0.015*outdoors + 0.014*nice',
 u'0.026*free + 0.023*active + 0.016*well + 0.015*professional + 0.014*good + 0.014*working + 0.014*educated + 0.012*future + 0.012*enjoy + 0.012*fit',
 u'0.017*lead + 0.015*blush + 0.015*one + 0.014*night + 0.012*take + 0.012*stand + 0.011*us + 0.009*connection + 0.009*checking + 0.008*everything',
 u'0.028*female + 0.022*kinky + 0.020*dream + 0.020*exploring + 0.018*box + 0.018*beautiful + 0.017*pleasure + 0.017*faithful + 0.015*sexua

# I try non-negative matrix factorisation to topic-model the profile abstracts here. It's a lot quicker than LSA or LDA.

In [118]:
tx = [" ".join(a) for a in texts]

In [120]:
from __future__ import print_function
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

n_samples = len(tx)
n_features = 5000
n_topics = 10
n_top_words = 20

t0 = time()
print("Loading dataset and extracting TF-IDF features...")


vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features)
tfidf = vectorizer.fit_transform(tx)
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model with n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
nmf = NMF(n_components=n_topics, random_state=1).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

feature_names = vectorizer.get_feature_names()

for topic_idx, topic in enumerate(nmf.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Loading dataset and extracting TF-IDF features...
done in 14.866s.
Fitting the NMF model with n_samples=496815 and n_features=5000...
done in 87.230s.
Topic #0:
paiddelete hi ladies meet guy say name nice chat people talk single site hope thanks checking interested girls friends everyone

Topic #1:
fun guy loving outgoing going nsa little drama likes strings wanna easy adventurous person lookin young safe ready clean times

Topic #2:
love women outdoors oral please sports woman give travel music play giving making also movies go going laugh pleasure adventure

Topic #3:
life enjoy excitement live short guy going little need spice person fullest passion great back adventure work loves easy hard

Topic #4:
new open anything things try minded people something trying willing pretty meet always almost experiences exciting much person meeting goes

Topic #5:
want know woman get please one women feel ask really man relationship find need interested wants take well married right

Topic #6:
loo